In [ ]:
# Install requirements
!wget https://github.com/eur-nl/bongaerts-10k-rag/raw/refs/heads/main/chroma.sqlite3
!pip install langchain-community
!pip install langchain-chroma
!pip install langchain-huggingface
!pip install langchain-ollama

In [2]:
# Import requirements
import os
import subprocess

from chromadb.config import Settings
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

In [12]:
"""
Pick a segment from the vector database for RAG. Possible values:
"500" - static chunks with 500 characters
"1000" - static chunks with 1000 characters
"2500" - static chunks with 2500 characters
"optimized" - dynamic optimized chunks
"""
db_segment = "2500"

In [4]:
# Get the database and embeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [ ]:
# Initialize the vector dabase
db_segments = ["500", "1000", "2500", "optimized"]

if db_segment not in db_segments:
    db_segment = "optimized"

vector_db = Chroma(
    persist_directory="/content",
    embedding_function=embeddings,
    collection_name=f"10k_{db_segment}"
)

In [ ]:
# Install and run ollama
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})
!curl https://ollama.ai/install.sh | sh
!nohup ollama serve > ollama.log 2>&1 &
!ollama pull llama3

In [24]:
llm = ChatOllama(model="llama3.2:1b")

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

In [ ]:
# Ask a question
query = "Which products are mentioned?"
print(chain.invoke(query))